<a href="https://www.kaggle.com/code/stutiiikatariaaa/artificialnn?scriptVersionId=270018287" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess data
df = pd.read_csv('/kaggle/input/solargeneration/dataset.csv')
df['date_time'] = pd.to_datetime(df['date_time'], format='%d-%m-%Y %H:%M')
df['hour'] = df['date_time'].dt.hour
df['day_of_week'] = df['date_time'].dt.dayofweek

In [ ]:
# Features and target
features = ['temperature', 'Ground_radiation_intensity ', 'hour']
X = df[features]
y = df['Photovoltaic_power_generation']

# Train-test split
train_size = int(len(X) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames
X_train = pd.DataFrame(X_train_scaled, columns=features, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=features, index=X_test.index)

In [ ]:
# Build ANN
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

In [ ]:
# Predict
y_pred = model.predict(X_test)

# Metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nANN Test Set Evaluation:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")


In [ ]:
# Plot actual vs predicted
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()}, index=y_test.index)
plt.figure(figsize=(18, 6))
results.head(500).plot(ax=plt.gca())
plt.title("ANN: Actual vs Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index")
plt.ylabel("Photovoltaic Power Generation")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
pip install optuna

In [ ]:
pip install optuna-integration[tfkeras]

In [ ]:
import optuna
from optuna.integration import TFKerasPruningCallback
from tensorflow.keras.callbacks import EarlyStopping

def create_model(trial):
    model = Sequential()

    # Suggest number of layers
    n_layers = trial.suggest_int('n_layers', 1, 3)

    input_dim = X_train.shape[1]
    for i in range(n_layers):
        num_hidden = trial.suggest_int(f'n_units_l{i}', 32, 256)
        activation = trial.suggest_categorical(f'act_l{i}', ['relu', 'tanh'])
        if i == 0:
            model.add(Dense(num_hidden, input_dim=input_dim, activation=activation))
        else:
            model.add(Dense(num_hidden, activation=activation))

    model.add(Dense(1))  # Output layer

    # Compile
    lr = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='mse', optimizer=optimizer)

    return model


def objective(trial):
    model = create_model(trial)

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]),
        verbose=0,
        callbacks=[early_stop]
    )

    # Evaluate on validation set
    val_loss = history.history['val_loss'][-1]
    return val_loss


In [ ]:
from tensorflow.keras.optimizers import Adam

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("\nBest Hyperparameters:")
print(study.best_params)

In [ ]:
# Use best parameters to build and train final model
best_trial = study.best_trial
model = create_model(best_trial)

model.fit(X_train, y_train,
          validation_split=0.1,
          epochs=50,
          batch_size=best_trial.params['batch_size'],
          verbose=0)

# Predict
y_pred = model.predict(X_test)

In [ ]:
# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nTuned ANN Test Set Evaluation:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()}, index=y_test.index)
plt.figure(figsize=(18, 6))
results.head(500).plot(ax=plt.gca())
plt.title("Tuned ANN: Actual vs Predicted (First 500 Samples)", fontsize=16)
plt.xlabel("Time Index")
plt.ylabel("Photovoltaic Power Generation")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
